# Generate automated session reports for all participants

In [ ]:
import pandas as pd
import numpy as np
import mne
import mne_bids
import glob
import warnings

import re
import subprocess

import os
import papermill as pm
warnings.filterwarnings('ignore')

from nbconvert import HTMLExporter
from nbconvert.preprocessors import ExecutePreprocessor
from nbformat import read


### Select task parameters and file paths

In [ ]:
project_path = "/project/def-emayada/q1k/experimental/"
pyll_path = "derivatives/pylossless/"
sync_loss_path = "derivatives/sync_loss/"
code_path = "code/q1k_sync_loss/"
out_path = ""
html_reports_path = "session_reports/"
task_id = "TO"
run_id = '1'
session_id = '01'


In [ ]:
# Sanity check to see which task you would like to make reports for 
print(task_id)

In [ ]:
# Generate list of session reports already processed so they are not processed again in next step
processed_sessions = []

for subject in glob.glob(project_path + pyll_path + sync_loss_path + code_path + html_reports_path + task_id + "/*.html"): 
    html_file = subject.split('/')[-1]
    processed_sessions.append(html_file.split('_')[:1])
    print(subject)

print('Existing session reports:')
processed_sessions 

pattern = project_path + pyll_path + '**/eeg/*' + task_id + '*.edf'
file_paths = glob.glob(pattern, recursive=True)


In [ ]:
# Function to extract details from filename
def extract_job_info(filename):
    # Define the pattern to capture the required sections
    pattern = r"sub-(.*?)_ses-(.*?)_task-(.*?)_run-(.*?)_eeg\.edf"
    match = re.match(pattern, filename)

    if match:
        # Extract the groups from the match
        subject_id = match.group(1)
        session_id = match.group(2)
        task_id = match.group(3)
        run_id = match.group(4)
        return subject_id, session_id, task_id, run_id
    else:
        raise ValueError("Filename pattern did not match.")


## Generate html session reports for all participants

In [ ]:
#glob.glob(project_path + pyll_path + '**/*.edf')

In [ ]:
# Make sure the output directory exists
if not os.path.exists(html_reports_path):
    os.makedirs(html_reports_path)

# Create a list of sessions with errors
error_subjects = []

#Loop through existing sessions and execute the q1k_generate_individual_reports.ipynb if a session report does not already exist
for file in file_paths:

    file_name = os.path.basename(file)
    print('Current data file: ' + file_name)
    ## Select anything after the Q1K and before the AEP
    subject_id = file_name.split("_")[0].split("-")[1]
    print('Participant ID: ' + subject_id)

    # Skip sessions that have a session report in the output directory
    print(subject_id)
    if any(subject_id in session for session in processed_sessions):
        print(subject_id + ' has already been processed')
        continue    

    # Define paths
    input_notebook = project_path + pyll_path + sync_loss_path + 'code/q1k_sync_loss/' + 'q1k_sync_loss.ipynb'

    # Make sure the directory exists
    if not os.path.exists(f'{project_path}{pyll_path}{sync_loss_path}{code_path}{html_reports_path}{task_id}/executed_notebooks/'):
        os.makedirs(f'{project_path}{pyll_path}{sync_loss_path}{code_path}{html_reports_path}{task_id}/executed_notebooks/')

    output_notebook = f'{project_path}{pyll_path}{sync_loss_path}{code_path}{html_reports_path}{task_id}/executed_notebooks/{subject_id}_{task_id}_executed.ipynb'
    print('Output notebook file: ' + output_notebook)
    output_html = f'{project_path}{pyll_path}{sync_loss_path}{code_path}{html_reports_path}{task_id}/{subject_id}_{task_id}.html'
    print('Output HTML file: ' + output_html)

    try:
        # Execute the notebook
        pm.execute_notebook(input_notebook, output_notebook, kernel_name = 'q1k_env', parameters=dict(subject_id=subject_id, task_id=task_id, run_id=run_id, session_id=session_id))

        # Convert executed notebook to HTML
        html_exporter = HTMLExporter()
        html_exporter.exclude_input = True

        (body, resources) = html_exporter.from_filename(output_notebook)

        # Save HTML output
        with open(output_html, 'w', encoding='utf-8') as f:
            f.write(body)

        print(f"HTML report saved for {subject_id}.")
    

    except Exception as e:
        # Handle the error 
        error_subjects.append(subject_id)
        print(f"Error while processing {subject_id}: {e}")

# Print out the list of subjects with errors
print( "These subjects have errors: " + str(error_subjects) + " and need to be reprocessed")